# Augmenting data for training by shifting IMU

# We generate augmented data with
- 50ms, 100ms, 150ms, 200ms, 500ms, and 1000ms of shifts.
- The shifting is done in the IMU modality.
- Training labels are are aligned with the Audio/Video Modality.
- The shift are saved in the end.

In [1]:
import numpy as np
import scipy.io

from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint


import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.core import Permute, Reshape
from keras import backend as K

from matplotlib import pyplot as plt
from PIL import Image as img_PIL


# random seed.
rand_seed = 2

from numpy.random import seed
seed(rand_seed)
from tqdm.keras import TqdmCallback
import tensorflow
tensorflow.random.set_seed(rand_seed)

## 1. Loading the data

In [2]:
path='C:/Users/macro/Desktop/UCLA Class/ECE 209AS ML/TimeAwareness/data/'

def get_train_and_validate_data(path=path):
    Train_data = np.load(path+'Data_train_all.npz')
    Labels = Train_data['arr_1']
    Features_imu = np.asarray(Train_data['arr_0'],dtype=np.float64)
    Features_audio = Train_data['arr_2']
    Features_video = Train_data['arr_3']
    
    randomize = np.arange(len(Labels))
    np.random.shuffle(randomize)
    
    Labels = Labels[randomize]
    Features_imu = Features_imu[randomize]
    Features_audio = Features_audio[randomize]
    Features_video = Features_video[randomize]
    
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 
    
    new_Labels = np.split(Labels,[Labels.shape[0] - 2000])
    new_Features_imu = np.split(Features_imu,[Features_imu.shape[0] - 2000])
    new_Features_audio = np.split(Features_audio,[Features_audio.shape[0] - 2000])
    new_Features_video = np.split(Features_video,[Features_video.shape[0] - 2000])
    
    return new_Labels[0],new_Features_imu[0],new_Features_audio[0],new_Features_video[0],new_Labels[1],new_Features_imu[1],new_Features_audio[1],new_Features_video[1]


def get_test_data(path=path):
    Train_data = np.load(path+'Data_test_71.pkl',allow_pickle=True)
    Labels = np.asarray(Train_data[1])
    Features_imu = np.asarray(Train_data[0],dtype=np.float64)
    Features_audio = Train_data[2]
    Features_video = Train_data[3]
    
    randomize = np.arange(len(Labels))
    np.random.shuffle(randomize)
    
    Labels = Labels[randomize]
    Features_imu = Features_imu[randomize]
    Features_audio = Features_audio[randomize]
    Features_video = Features_video[randomize]
    
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 

    return Labels,Features_imu,Features_audio,Features_video

In [3]:
Labels,Features_imu,Features_audio,Features_video,Labels2,Features_imu2,Features_audio2,Features_video2 = get_train_and_validate_data()
Labels3,Features_imu3,Features_audio3,Features_video3 = get_test_data()

print(Labels.shape,Features_imu.shape,Features_audio.shape,Features_video.shape)
print(Labels2.shape,Features_imu2.shape,Features_audio2.shape,Features_video2.shape)
print(Labels3.shape,Features_imu3.shape,Features_audio3.shape,Features_video3.shape)

(9976,) (9976, 1, 40, 12) (9976, 193) (9976, 45, 64, 64, 3)
(2000,) (2000, 1, 40, 12) (2000, 193) (2000, 45, 64, 64, 3)
(1377,) (1377, 1, 40, 12) (1377, 193) (1377, 45, 64, 64, 3)


## Observation: All of these test labels are continuous.
### Time window used to create a sample: 2 Second.

# Time shifting, our goal is to use the longest windows and insert time shifting into them

# First preprocess the data to get the continuous acc and gyro windows
- data ordering: 12 sensors each with 40 samples.
- 12 sensors are: acc_right, gyro_right, acc_left, gyro_left

In [4]:
# We see all of the labels are continuous

Labels = np.array(Labels)
Labels2 = np.array(Labels2)
Labels3 = np.array(Labels3)

print(Labels[:100])
print(Labels2[:100])
print(Labels3[:100])

['handwashing' 'jump' 'downstair' 'exercise' 'jump' 'handwashing' 'jump'
 'walk' 'run' 'upstair' 'exercise' 'upstair' 'upstair' 'upstair' 'upstair'
 'walk' 'upstair' 'exercise' 'jump' 'exercise' 'downstair' 'jump'
 'exercise' 'downstair' 'upstair' 'run' 'run' 'exercise' 'exercise'
 'upstair' 'downstair' 'exercise' 'downstair' 'jump' 'handwashing'
 'downstair' 'upstair' 'run' 'handwashing' 'jump' 'run' 'walk' 'run'
 'handwashing' 'jump' 'downstair' 'downstair' 'walk' 'downstair' 'upstair'
 'upstair' 'handwashing' 'jump' 'downstair' 'downstair' 'upstair' 'jump'
 'upstair' 'upstair' 'downstair' 'jump' 'downstair' 'downstair' 'jump'
 'upstair' 'downstair' 'downstair' 'jump' 'upstair' 'downstair'
 'handwashing' 'handwashing' 'run' 'walk' 'upstair' 'downstair' 'walk'
 'downstair' 'upstair' 'downstair' 'handwashing' 'handwashing' 'run'
 'upstair' 'handwashing' 'jump' 'handwashing' 'run' 'downstair' 'jump'
 'jump' 'run' 'exercise' 'run' 'upstair' 'handwashing' 'downstair' 'run'
 'jump' 'exerci

In [5]:
# Reorder the labels so they are continuous

In [6]:
def number_labels(y_data):
    Mapping=dict()
    sub_dirs=['downstair','upstair','run','jump','walk','handwashing','exercise']

    categories=10

    count=0
    for i in sub_dirs:
        Mapping[i]=count
        count=count+1

    y_features2=[]
    for i in range(len(y_data)):
        Type=y_data[i]
        lab=Mapping[Type]
        y_features2.append(lab)

    y_features=np.array(y_features2, dtype=np.int32)
    #y_features=y_features.reshape(y_features.shape[0],1)
    return y_features.astype(np.int32)

In [7]:
Labels = number_labels(Labels)
Labels2 = number_labels(Labels2)
Labels3 = number_labels(Labels3)
    
print(Labels[:10])
print(Labels2[:10])
print(Labels3[:10])

[5 3 0 6 3 5 3 4 2 1]
[0 5 2 4 4 0 1 0 3 2]
[5 3 1 1 1 0 1 6 0 1]


In [8]:
# Get a sorting order for the labels
train_order = np.argsort(Labels)
validate_order = np.argsort(Labels2)
test_order = np.argsort(Labels3)
    
print(Labels[train_order])
print(Labels2[validate_order])
print(Labels3[test_order])

[0 0 0 ... 6 6 6]
[0 0 0 ... 6 6 6]
[0 0 0 ... 6 6 6]


# Use this sorting order to recreate the array

In [9]:
Train_imu = []
Train_labels = []
Train_audio = []
Train_video = []
        
for i in range(Labels.shape[0]):
    Train_imu.append(Features_imu[train_order[i]])
    Train_labels.append(Labels[train_order[i]])
    Train_audio.append(Features_audio[train_order[i]])
    Train_video.append(Features_video[train_order[i]])

In [10]:
Validate_imu = []
Validate_labels = []
Validate_audio = []
Validate_video = []
        
for i in range(Labels2.shape[0]):
    Validate_imu.append(Features_imu2[validate_order[i]])
    Validate_labels.append(Labels2[validate_order[i]])
    Validate_audio.append(Features_audio2[validate_order[i]])
    Validate_video.append(Features_video2[validate_order[i]])

In [11]:
Test_imu = []
Test_labels = []
Test_audio = []
Test_video = []
        
for i in range(Labels3.shape[0]):
    Test_imu.append(Features_imu3[test_order[i]])
    Test_labels.append(Labels3[test_order[i]])
    Test_audio.append(Features_audio3[test_order[i]])
    Test_video.append(Features_video3[test_order[i]])

In [12]:
Train_imu = np.array(Train_imu,dtype=np.float64)
Train_labels = np.array(Train_labels)
Train_audio = np.array(Train_audio)
Train_video = np.array(Train_video)
    
Validate_imu = np.array(Validate_imu,dtype=np.float64)
Validate_labels = np.array(Validate_labels)
Validate_audio = np.array(Validate_audio)
Validate_video = np.array(Validate_video)
    
Test_imu = np.array(Test_imu,dtype=np.float64)
Test_labels = np.array(Test_labels)
Test_audio = np.array(Test_audio)
Test_video = np.array(Test_video)   

print(Train_imu.shape, Train_labels.shape, Train_audio.shape, Train_video.shape)
print(Validate_imu.shape, Validate_labels.shape, Validate_audio.shape, Validate_video.shape)
print(Test_imu.shape, Test_labels.shape, Test_audio.shape, Test_video.shape)

(9976, 1, 40, 12) (9976,) (9976, 193) (9976, 45, 64, 64, 3)
(2000, 1, 40, 12) (2000,) (2000, 193) (2000, 45, 64, 64, 3)
(1377, 1, 40, 12) (1377,) (1377, 193) (1377, 45, 64, 64, 3)


In [13]:
print(Train_labels)
print(Validate_labels)
print(Test_labels)

[0 0 0 ... 6 6 6]
[0 0 0 ... 6 6 6]
[0 0 0 ... 6 6 6]


In [14]:
from keras.utils import to_categorical

Train_labels = to_categorical(Train_labels)
Validate_labels = to_categorical(Validate_labels)
Test_labels = to_categorical(Test_labels)

In [15]:
#test accuracy of model on the newly concatenated data so we don't mess up the things

from keras.models import load_model
model_path = 'Baseline Models/imu_video_combined_model_checkpoint'
model = load_model(model_path)


model.evaluate([Train_imu,Train_video],Train_labels)
model.evaluate([Validate_imu,Validate_video],Validate_labels)
model.evaluate([Test_imu,Test_video],Test_labels)

44/44 [==============================] - 4s 101ms/step - loss: 0.2310 - accuracy: 0.9528


[0.23102378845214844, 0.9527959227561951]

# Testing accuracy on Reshuffled data.
## Note: not time errors are introduced so Accuracy should be same as the fusion model on original continuous test data

# Now aggregating the data and doing the timing errors

In [16]:
Train_acc_right = []
Train_gyro_right = []
Train_acc_left = []
Train_gyro_left = []

for i in range(Train_imu.shape[0]):
    Train_acc_right.append([Train_imu[i,0,:,0],Train_imu[i,0,:,1],Train_imu[i,0,:,2]])
    Train_gyro_right.append([Train_imu[i,0,:,3],Train_imu[i,0,:,4],Train_imu[i,0,:,5]])
    Train_acc_left.append([Train_imu[i,0,:,6],Train_imu[i,0,:,7],Train_imu[i,0,:,8]])
    Train_gyro_left.append([Train_imu[i,0,:,9],Train_imu[i,0,:,10],Train_imu[i,0,:,11]])
        

In [17]:
Validate_acc_right = []
Validate_gyro_right = []
Validate_acc_left = []
Validate_gyro_left = []

for i in range(Validate_imu.shape[0]):
    Validate_acc_right.append([Validate_imu[i,0,:,0],Validate_imu[i,0,:,1],Validate_imu[i,0,:,2]])
    Validate_gyro_right.append([Validate_imu[i,0,:,3],Validate_imu[i,0,:,4],Validate_imu[i,0,:,5]])
    Validate_acc_left.append([Validate_imu[i,0,:,6],Validate_imu[i,0,:,7],Validate_imu[i,0,:,8]])
    Validate_gyro_left.append([Validate_imu[i,0,:,9],Validate_imu[i,0,:,10],Validate_imu[i,0,:,11]])

In [18]:
Test_acc_right = []
Test_gyro_right = []
Test_acc_left = []
Test_gyro_left = []

for i in range(Test_imu.shape[0]):
    Test_acc_right.append([Test_imu[i,0,:,0],Test_imu[i,0,:,1],Test_imu[i,0,:,2]])
    Test_gyro_right.append([Test_imu[i,0,:,3],Test_imu[i,0,:,4],Test_imu[i,0,:,5]])
    Test_acc_left.append([Test_imu[i,0,:,6],Test_imu[i,0,:,7],Test_imu[i,0,:,8]])
    Test_gyro_left.append([Test_imu[i,0,:,9],Test_imu[i,0,:,10],Test_imu[i,0,:,11]])
        

In [19]:

Train_acc_right = np.array(Train_acc_right)
Train_gyro_right = np.array(Train_gyro_right)
Train_acc_left = np.array(Train_acc_left)
Train_gyro_left = np.array(Train_gyro_left)
    
Validate_acc_right = np.array(Validate_acc_right)
Validate_gyro_right = np.array(Validate_gyro_right)
Validate_acc_left = np.array(Validate_acc_left)
Validate_gyro_left = np.array(Validate_gyro_left)

Test_acc_right = np.array(Test_acc_right)
Test_gyro_right = np.array(Test_gyro_right)
Test_acc_left = np.array(Test_acc_left)
Test_gyro_left = np.array(Test_gyro_left)
    
print(Train_acc_right.shape)
print(Train_gyro_right.shape)
print(Train_acc_left.shape)
print(Train_gyro_left.shape)
    
print(Validate_acc_right.shape)
print(Validate_gyro_right.shape)
print(Validate_acc_left.shape)
print(Validate_gyro_left.shape)

print(Test_acc_right.shape)
print(Test_gyro_right.shape)
print(Test_acc_left.shape)
print(Test_gyro_left.shape)

(9976, 3, 40)
(9976, 3, 40)
(9976, 3, 40)
(9976, 3, 40)
(2000, 3, 40)
(2000, 3, 40)
(2000, 3, 40)
(2000, 3, 40)
(1377, 3, 40)
(1377, 3, 40)
(1377, 3, 40)
(1377, 3, 40)


In [20]:
Train_acc_right_cont = Train_acc_right[0,]
Train_gyro_right_cont = Train_gyro_right[0,]
Train_acc_left_cont = Train_acc_left[0,]
Train_gyro_left_cont = Train_gyro_left[0,]

for i in range(1, Features_imu.shape[0]):
    Train_acc_right_cont = np.hstack((Train_acc_right_cont,Train_acc_right[i,]))
    Train_gyro_right_cont = np.hstack((Train_gyro_right_cont,Train_gyro_right[i,]))
    Train_acc_left_cont = np.hstack((Train_acc_left_cont,Train_acc_left[i,]))
    Train_gyro_left_cont = np.hstack((Train_gyro_left_cont,Train_gyro_left[i,]))

print('done')

done


In [21]:
Validate_acc_right_cont = Validate_acc_right[0,]
Validate_gyro_right_cont = Validate_gyro_right[0,]
Validate_acc_left_cont = Validate_acc_left[0,]
Validate_gyro_left_cont = Validate_gyro_left[0,]


for i in range(1, Features_imu2.shape[0]):
    Validate_acc_right_cont = np.hstack((Validate_acc_right_cont,Validate_acc_right[i,]))
    Validate_gyro_right_cont = np.hstack((Validate_gyro_right_cont,Validate_gyro_right[i,]))
    Validate_acc_left_cont = np.hstack((Validate_acc_left_cont,Validate_acc_left[i,]))
    Validate_gyro_left_cont = np.hstack((Validate_gyro_left_cont,Validate_gyro_left[i,]))

print('done')

done


In [22]:
Test_acc_right_cont = Test_acc_right[0,]
Test_gyro_right_cont = Test_gyro_right[0,]
Test_acc_left_cont = Test_acc_left[0,]
Test_gyro_left_cont = Test_gyro_left[0,]

for i in range(1, Features_imu3.shape[0]):
    Test_acc_right_cont = np.hstack((Test_acc_right_cont,Test_acc_right[i,]))
    Test_gyro_right_cont = np.hstack((Test_gyro_right_cont,Test_gyro_right[i,]))
    Test_acc_left_cont = np.hstack((Test_acc_left_cont,Test_acc_left[i,]))
    Test_gyro_left_cont = np.hstack((Test_gyro_left_cont,Test_gyro_left[i,]))

print('done')

done


In [23]:
print(Train_acc_right_cont.shape)
print(Train_gyro_right_cont.shape)
print(Train_acc_left_cont.shape)
print(Train_gyro_left_cont.shape)
    

print(Validate_acc_right_cont.shape)
print(Validate_gyro_right_cont.shape)
print(Validate_acc_left_cont.shape)
print(Validate_gyro_left_cont.shape)
    

print(Test_acc_right_cont.shape)
print(Test_gyro_right_cont.shape)
print(Test_acc_left_cont.shape)
print(Test_gyro_left_cont.shape)

(3, 399040)
(3, 399040)
(3, 399040)
(3, 399040)
(3, 80000)
(3, 80000)
(3, 80000)
(3, 80000)
(3, 55080)
(3, 55080)
(3, 55080)
(3, 55080)


# Now shifting the samples

# Defining function to do the evaluation

In [24]:
def get_data_sample_shift(acc_right_cont, gyro_right_cont, acc_left_cont, gyro_left_cont, audio, labels, video, shift_samples = 1):
    sample_size = 40 #need to be 40, as decided during training
    total_samples = acc_right_cont.shape[1]
    
    #print(total_samples)
    
    current_cursor = shift_samples

    i = 0

    acc_right_pro= np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    #print(acc_right_pro.shape)

    gyro_right_pro= np.array(gyro_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    acc_left_pro= np.array(acc_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    gyro_left_pro= np.array(gyro_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    #print(np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size]).shape)

    while current_cursor<=(total_samples-2*sample_size):
        current_cursor = current_cursor + sample_size
        #print(current_cursor," : ", i)
        a=acc_right_pro
        b=np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        acc_right_pro = np.concatenate((a,b),axis=0)


        a=gyro_right_pro
        b=np.array(gyro_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        gyro_right_pro = np.concatenate((a,b),axis=0)


        a=acc_left_pro
        b=np.array(acc_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        #print(a.shape,b.shape)
        acc_left_pro = np.concatenate((a,b),axis=0)

        a=gyro_left_pro
        b=np.array(gyro_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        gyro_left_pro = np.concatenate((a,b),axis=0)
        i = i+1
        
    IMU_processed = np.concatenate((acc_right_pro,gyro_right_pro,acc_left_pro,gyro_left_pro),axis=1)
    IMU_processed = IMU_processed[:,np.newaxis,:,:]
    IMU_processed = np.swapaxes(IMU_processed,2,3)
        
    size = IMU_processed.shape[0]    
        
    return audio[:size],IMU_processed, labels[:size], video[:size]

# Generating more training datasets by doing shifting
- 1 sample shift = 50 ms timing error.
- 20 sample shift = 1000ms timing error.
- 1000ms augmentation uses shifts (1,2,10,20)

In [25]:
#shift by 1 sample
Train_Features_audio_1, Train_IMU_1 , Train_Labels_1, Train_Features_video_1 = get_data_sample_shift(Train_acc_right_cont, Train_gyro_right_cont, Train_acc_left_cont, Train_gyro_left_cont, Features_audio, Train_labels, Features_video, shift_samples = 1)
Validate_Features_audio_1, Validate_IMU_1 , Validate_Labels_1, Validate_Features_video_1 = get_data_sample_shift(Validate_acc_right_cont, Validate_gyro_right_cont, Validate_acc_left_cont, Validate_gyro_left_cont, Features_audio2, Validate_labels, Features_video2, shift_samples = 1)

print(Train_Features_audio_1.shape, Train_IMU_1.shape, Train_Labels_1.shape, Train_Features_video_1.shape)
print(Validate_Features_audio_1.shape, Validate_IMU_1.shape, Validate_Labels_1.shape, Validate_Features_video_1.shape)

(9975, 193) (9975, 1, 40, 12) (9975, 7) (9975, 45, 64, 64, 3)
(1999, 193) (1999, 1, 40, 12) (1999, 7) (1999, 45, 64, 64, 3)


In [26]:
import numpy as np
path_1_Train = 'E:/augmented_data/train_data_1_shift'
path_1_Validate = 'E:/augmented_data/validate_data_1_shift'

np.savez(path_1_Train, Train_IMU_1, Train_Labels_1, Train_Features_audio_1, Train_Features_video_1)
np.savez(path_1_Validate, Validate_IMU_1, Validate_Labels_1, Validate_Features_audio_1, Validate_Features_video_1)

In [27]:
#shift by 2 sample
Train_Features_audio_2, Train_IMU_2 , Train_Labels_2, Train_Features_video_2 = get_data_sample_shift(Train_acc_right_cont, Train_gyro_right_cont, Train_acc_left_cont, Train_gyro_left_cont, Features_audio, Train_labels, Features_video, shift_samples = 2)
Validate_Features_audio_2, Validate_IMU_2 , Validate_Labels_2, Validate_Features_video_2 = get_data_sample_shift(Validate_acc_right_cont, Validate_gyro_right_cont, Validate_acc_left_cont, Validate_gyro_left_cont, Features_audio2, Validate_labels, Features_video2, shift_samples = 2)

print(Train_Features_audio_2.shape, Train_IMU_2.shape, Train_Labels_2.shape, Train_Features_video_2.shape)
print(Validate_Features_audio_2.shape, Validate_IMU_2.shape, Validate_Labels_2.shape, Validate_Features_video_2.shape)

(9975, 193) (9975, 1, 40, 12) (9975, 7) (9975, 45, 64, 64, 3)
(1999, 193) (1999, 1, 40, 12) (1999, 7) (1999, 45, 64, 64, 3)


In [28]:
import numpy as np
path_2_Train = 'E:/augmented_data/train_data_2_shift'
path_2_Validate = 'E:/augmented_data/validate_data_2_shift'

np.savez(path_2_Train, Train_IMU_2, Train_Labels_2, Train_Features_audio_2, Train_Features_video_2)
np.savez(path_2_Validate, Validate_IMU_2, Validate_Labels_2, Validate_Features_audio_2, Validate_Features_video_2)

In [29]:
#shift by 10 sample
Train_Features_audio_10, Train_IMU_10 , Train_Labels_10, Train_Features_video_10 = get_data_sample_shift(Train_acc_right_cont, Train_gyro_right_cont, Train_acc_left_cont, Train_gyro_left_cont, Features_audio, Train_labels, Features_video, shift_samples = 10)
Validate_Features_audio_10, Validate_IMU_10 , Validate_Labels_10, Validate_Features_video_10 = get_data_sample_shift(Validate_acc_right_cont, Validate_gyro_right_cont, Validate_acc_left_cont, Validate_gyro_left_cont, Features_audio2, Validate_labels, Features_video2, shift_samples = 10)

print(Train_Features_audio_10.shape, Train_IMU_10.shape, Train_Labels_10.shape, Train_Features_video_10.shape)
print(Validate_Features_audio_10.shape, Validate_IMU_10.shape, Validate_Labels_10.shape, Validate_Features_video_10.shape)

(9975, 193) (9975, 1, 40, 12) (9975, 7) (9975, 45, 64, 64, 3)
(1999, 193) (1999, 1, 40, 12) (1999, 7) (1999, 45, 64, 64, 3)


In [30]:
import numpy as np
path_10_Train = 'E:/augmented_data/train_data_10_shift'
path_10_Validate = 'E:/augmented_data/validate_data_10_shift'

np.savez(path_10_Train, Train_IMU_10, Train_Labels_10, Train_Features_audio_10, Train_Features_video_10)
np.savez(path_10_Validate, Validate_IMU_10, Validate_Labels_10, Validate_Features_audio_10, Validate_Features_video_10)

In [31]:
#shift by 20 sample
Train_Features_audio_20, Train_IMU_20 , Train_Labels_20, Train_Features_video_20 = get_data_sample_shift(Train_acc_right_cont, Train_gyro_right_cont, Train_acc_left_cont, Train_gyro_left_cont, Features_audio, Train_labels, Features_video, shift_samples = 20)
Validate_Features_audio_20, Validate_IMU_20 , Validate_Labels_20, Validate_Features_video_20 = get_data_sample_shift(Validate_acc_right_cont, Validate_gyro_right_cont, Validate_acc_left_cont, Validate_gyro_left_cont, Features_audio2, Validate_labels, Features_video2, shift_samples = 20)

print(Train_Features_audio_20.shape, Train_IMU_20.shape, Train_Labels_20.shape, Train_Features_video_20.shape)
print(Validate_Features_audio_20.shape, Validate_IMU_20.shape, Validate_Labels_20.shape, Validate_Features_video_20.shape)

(9975, 193) (9975, 1, 40, 12) (9975, 7) (9975, 45, 64, 64, 3)
(1999, 193) (1999, 1, 40, 12) (1999, 7) (1999, 45, 64, 64, 3)


In [32]:
import numpy as np
path_20_Train = 'E:/augmented_data/train_data_20_shift'
path_20_Validate = 'E:/augmented_data/validate_data_20_shift'

np.savez(path_20_Train, Train_IMU_20, Train_Labels_20, Train_Features_audio_20, Train_Features_video_20)
np.savez(path_20_Validate, Validate_IMU_20, Validate_Labels_20, Validate_Features_audio_20, Validate_Features_video_20)

In [33]:
for i in range(1,41):
    Test_Features_audio_shiftAmount,Test_IMU_shiftAmount, Test_Labels_shiftAmount,Test_Features_video_shiftAmount = get_data_sample_shift(Test_acc_right_cont, Test_gyro_right_cont, Test_acc_left_cont, Test_gyro_left_cont, Features_audio3, Test_labels, Features_video3, shift_samples = i)
    path_shiftAmount_Test = 'E:/augmented_data/test_data_'+str(i)+'_shift'
    np.savez(path_shiftAmount_Test, Test_IMU_shiftAmount, Test_Labels_shiftAmount, Test_Features_audio_shiftAmount, Test_Features_video_shiftAmount)